In [1]:
import json
from img_doc.extractors.word_extractors import BaseWordExtractor
import numpy as np
from typing import List
from img_doc.data_structures import Word, Block, Image
import pytesseract
import cv2

In [2]:
path_file_train = "/home/daniil/program/dataset/publaynet/train.json"
with open(path_file_train, "r") as f:
    print("ok")


COUNT_MIN_CATEGORY = 1000
images_start = 1
annotations_start = 27417937
category_1000 = 52300000
# 1000 категорий на category_1000 от annotations_start
category_start = 1696001803

ok


In [3]:
def my_read(seek=0, step=2000):
    with open(path_file_train, "r") as f:
        f.seek(seek)
        str_ = f.read(step)
    return str_
    

In [4]:

class TesseractWordExtractor(BaseWordExtractor):
    def extract_from_img(self, img: np) -> List[Word]:
        k = 4
        dim = (k*img.shape[1], k*img.shape[0])
        img_ = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
      
        tesseract_bboxes = pytesseract.image_to_data(
            config="-l eng",
            image=img_,
            output_type=pytesseract.Output.DICT)
        word_list = []
        
        for index_bbox, level in enumerate(tesseract_bboxes["level"]):
            if level == 5:
                word = Word(text = tesseract_bboxes["text"][index_bbox])
                word.set_point_and_size({
                    "x_top_left": round(tesseract_bboxes["left"][index_bbox]/k),
                    "y_top_left": round(tesseract_bboxes["top"][index_bbox]/k),
                    "width": round(tesseract_bboxes["width"][index_bbox]/k),
                    "height": round(tesseract_bboxes["height"][index_bbox]/k),
                })
                word_list.append(word)
        return word_list

tess = TesseractWordExtractor()

# Поиск нужного количества сегментов

In [5]:
step = 1000
min_size_dataset = COUNT_MIN_CATEGORY
key_category = '"category_id"'
key_image_id = '"image_id"'
len_key_category = 3+len(key_category)
list_count_category = {1: 0, 2: 0, 3: 0, 4: 0, 5: 0}
my_dataset = []
seek = annotations_start
while seek < category_start:    
    str_ = my_read(seek=seek, step=step)
    # try:
    index_char_category = str_.find(key_category)
    rigth_border = seek+index_char_category+len_key_category 
    if index_char_category != -1:
        str_info = my_read(rigth_border-step, step=step)
        index_char_image = str_info.rfind(key_image_id)
        rez = json.loads("{" + str_info[index_char_image:] + "}")
        my_dataset.append(rez)  
        list_count_category[rez["category_id"]] += 1
        
        stop_ = True
        for key, count in list_count_category.items():
            if count < min_size_dataset:
                stop_ = False

        if stop_:
            break
    
    seek = rigth_border  

In [6]:
list_count_category

{1: 52120, 2: 14087, 3: 1000, 4: 2691, 5: 2401}

# Поиск нужных картинок

In [7]:
list_image_id = list(set([row["image_id"] for row in my_dataset]))

In [8]:
step = 300

key_filename = '"file_name"'
list_exit = dict()
for key in list_image_id:
    list_exit[key] = False
    

my_images = []
seek = images_start
while seek < annotations_start:    
    str_ = my_read(seek=seek, step=step)
    # try:
    index_char_filename = str_.find(key_filename)
    left_border = seek+index_char_filename-1
    if index_char_filename != -1:
        str_info = my_read(left_border, step=step)
        right_bord = str_info.find("}")
        rez = json.loads(str_info[:right_bord+1])
        if rez["id"] in list_exit.keys():
            if not list_exit[rez["id"]]: 
                list_exit[rez["id"]] = True
                my_images.append(rez)
        for key, ind in list_exit.items():
            if ind:
                break

    
    seek = left_border+right_bord 

In [9]:
print(f"кол-во изображений:\t {len(my_images)}")
print(f"кол-во сегментов:\t {len(my_dataset)}")

кол-во изображений:	 7514
кол-во сегментов:	 72299


In [10]:
path_dir_image = "/home/daniil/program/dataset/publaynet/train"
path_mini_publaynet = "/home/daniil/program/dataset/micro_ts_publaynet"
path_mini_publaynet_train = "/home/daniil/program/dataset/micro_ts_publaynet/train"

In [11]:
import os, shutil

In [14]:
img_len = len(my_images)
if not os.path.exists(path_mini_publaynet):
    os.mkdir(path_mini_publaynet)
    os.mkdir(path_mini_publaynet_train)
for i, img in enumerate(my_images):
    pr = i/img_len*100
    print(f"    {pr:.2f}%    ", end="\r" )
    img_path = os.path.join(path_dir_image, img["file_name"])
    shutil.copy(
        img_path,
        os.path.join(path_mini_publaynet_train)
    )
    image = Image()
    image.set_img_from_path(img_path)
    words = tess.extract_from_img(image.img)
    json_words = [w.to_dict()  for w in words]
    with open(os.path.join(path_mini_publaynet_train, img["file_name"]+".json"), 'w') as f:
        json.dump({"words": json_words}, f)

In [15]:
categories = [{"supercategory": "", "id": 1, "name": "text"}, 
               {"supercategory": "", "id": 2, "name": "title"}, 
               {"supercategory": "", "id": 3, "name": "list"}, 
               {"supercategory": "", "id": 4, "name": "table"}, 
               {"supercategory": "", "id": 5, "name": "figure"}]

In [16]:
with open(os.path.join(path_mini_publaynet, "train.json"), 'w') as f:
    json.dump({"images": my_images, "annotations": my_dataset, "categories": categories}, f)